In [1]:
import os
import urllib.request
import re

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f: #打開檔案，使用讀取模式(r)，用UTF-8編碼讀取所有內容
    raw_text = f.read()

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_words = sorted(set(preprocessed))     #用set()把preprocessed包成集合，把重複的字詞(元)刪掉，再用sorted()案字母排序
#vocab_size = len(all_words)
vocab = {token:integer for integer,token in enumerate(all_words)} 

#token是all_words裡面被我們排序過的字詞(元)；integer是他們的編號


for i, item in enumerate(vocab.items()):
    if i < 10:
        print(item)

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)


## 2.2

In [2]:
import re

In [3]:
text = "Hello world. My name is Redfish."
output = re.split(r'(\s)', text)
print(output)

['Hello', ' ', 'world.', ' ', 'My', ' ', 'name', ' ', 'is', ' ', 'Redfish.']


In [4]:
text = "Hello world. My name is Redfish."
output = re.split(r'([,.]|\s)', text)
print(output)

['Hello', ' ', 'world', '.', '', ' ', 'My', ' ', 'name', ' ', 'is', ' ', 'Redfish', '.', '']


In [5]:
output = [item for item in output if item.strip()]
print(output)

['Hello', 'world', '.', 'My', 'name', 'is', 'Redfish', '.']


In [6]:
text = "Hello world. My name is Redfish."
output = re.split(r'([,.]|\s)', text)
output = [item for item in output if item.strip()]
print(output)

['Hello', 'world', '.', 'My', 'name', 'is', 'Redfish', '.']


## 2.3
#### 製作一個簡單的斷詞器


In [7]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab                             #將詞彙表儲存到self.str_to_int
        self.int_to_str = {i: s for s, i in vocab.items()}  #反向詞彙表

    #token -> token ID
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    #token ID -> token
    def decode(self, ids):
        text = " ".join(self.int_to_str[i] for i in ids)  #用空格把token串起來
        text = re.sub(r'\s([,.?_!"()\'])', r'\1', text)     #移除標點符號前的空格
        return text

In [8]:
tokenizer = SimpleTokenizer(vocab)
text = "It's the last he painted, you know." 
ids = tokenizer.encode(text)

print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 7]


In [9]:
'''text = "Hello, do you like tea?"
print(tokenizer.encode(text))'''

'text = "Hello, do you like tea?"\nprint(tokenizer.encode(text))'

## 2.4
#### 目的是加入特殊的token

1. |unk|:如果文本遇見未知的字詞時先用unk註記
2. |endoftext|:多個文本在訓練時，透過此token把不同文本區分開來

In [10]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}


In [11]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [12]:
class SimpleTokenizerv2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]    #未知的詞用<|unk|>代替
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join(self.int_to_str[i] for i in ids) 
        text = re.sub(r'\s+([,.?_!"()\'])', r'\1', text)
        return text
    
        

In [13]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = "<|endoftext|> ".join((text1, text2))
print(text)


Hello, do you like tea?<|endoftext|> In the sunlit terraces of the palace.


In [14]:
tokenizer = SimpleTokenizerv2(vocab)
ids = tokenizer.encode(text)

print(ids)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [15]:
text = tokenizer.decode(ids)

print(text)

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


不過GPT不會用<|unk|>標示新詞彙，是使用字節對編碼(Byte Pair Encoding, BPE)斷詞；是種把字詞分割成更小單位的斷詞法

## 2.5
#### BPE

In [16]:
#先確認自己有下載成功..
from importlib.metadata import version
import tiktoken 
print("tiketoken version:", version("tiktoken"))

tiketoken version: 0.12.0


In [17]:
tokenizer = tiktoken.get_encoding("gpt2")

In [18]:
#先看看token -> token ID
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"}) #將"<|endoftext|>"視為完整的特殊token
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [19]:
#token ID -> token
ids = tokenizer.decode(integers)
print(ids)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


有兩個重點
1. `<|endoftext|>` 在詞彙表裡面是最後一個ID
2. BPE可以正確的幫someunknownPlace編碼和解碼

因為BPE可以把未知單字拆成一個又一個的token，所以`[2114, 286, 617, 34680, 27271]`都是代表`someunknownPlace`

In [20]:
#practice

#將Akwirwier編碼並print出token ID
text = "Akwirwier"
integers = tokenizer.encode(text)
print(integers)

#確認是否可由這些token重建回Akwirwier
ids = tokenizer.decode(integers)
print(ids)

[33901, 86, 343, 86, 959]
Akwirwier


## 2.6
#### 使用滑動視窗法進行資料取樣

現在要做的是生成拿來訓練LLM的資料:`[輸入-目標]`的文本配對組合

In [21]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [22]:
#先取50個token後的東西當sample
enc_sample = enc_text[50:]

In [23]:
#建立兩個變數'x'和'y'，x為輸入多個token，y是把輸入的token往右偏一個位置後同數量的token作為預測目標
context_size = 4
x = enc_sample[0:context_size]
y = enc_sample[0:context_size+1]

x_ids = tokenizer.decode(x)
y_ids = tokenizer.decode(y)

print(f"x:{x}")
print(f"y:{y}")

print(f"x(decoded):{x_ids}")
print(f"y(decoded):{y_ids}")

x:[290, 4920, 2241, 287]
y:[290, 4920, 2241, 287, 257]
x(decoded): and established himself in
y(decoded): and established himself in a


In [24]:
for i in range(1, context_size+1):
    context = enc_sample[0:i]
    desired = enc_sample[i]
    print(context, "->", desired)

[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [25]:
#給我說人話喔
for i in range(1, context_size+1):
    context = enc_sample[0:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "->", tokenizer.decode([desired]))

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


建立好`[輸入目標對]`左邊是我們輸入的文本，右邊是他應該預測的詞彙

-----------------------

現在要搞一個載入器；他可以批次處理整個輸入的訓練集，並把輸入資料和目標資料轉換成**PyTorch張量**(多維陣列)  
PS. 想了解**pytorch**請參看**beginner_for_pytorch**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# 把資料集切成一塊一塊(批次輸入)
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chuck = token_ids[i:i + max_length]
            target_chuck = token_ids[i+1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chuck))
            self.target_ids.append(torch.tensor(target_chuck))    

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [27]:
# 建立可批次處理輸入與目標的資料載入器
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last,
                            num_workers=0)
    return dataloader

In [28]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1,
                                  max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [29]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [35]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Input:\n", inputs)
print("==========================================")
print("Target:\n", targets)

Input:
 tensor([[   11,   319,  1401,    77],
        [  379,   790,   966,   262],
        [  503,   287,   262, 37090],
        [ 9174,   262,  4286,  1871],
        [   26,   290,  1719,    11],
        [    0,  1375,  1807,   340],
        [ 4808,  5562,    62,   465],
        [  340,  1392,   284,   307]])
Target:
 tensor([[  319,  1401,    77,  3929],
        [  790,   966,   262,  3512],
        [  287,   262, 37090,   257],
        [  262,  4286,  1871,   607],
        [  290,  1719,    11,   319],
        [ 1375,  1807,   340,   262],
        [ 5562,    62,   465,  2106],
        [ 1392,   284,   307,  2081]])


## 2.7

把token ID換成嵌入向量

In [38]:
input_ids = torch.tensor([2, 3, 5, 1])

In [39]:
vocab_size = 6
output_dim = 3

In [40]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [42]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [43]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## 2.8

位置資訊的編碼

In [45]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [47]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [49]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
